In [1]:
from IPython.core.display import HTML

import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from sklearn.decomposition import TruncatedSVD

import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

import graphlab
from copy import copy, deepcopy

In [5]:
HTML("""<iframe
          width="600"
          height="400"
          seamless
          frameBorder="0"
          scrolling="no"
          src="http://localhost:8088/superset/explore/table/6/?form_data=%7B%22datasource%22%3A%226__table%22%2C%22viz_type%22%3A%22world_map%22%2C%22slice_id%22%3A19%2C%22granularity_sqla%22%3Anull%2C%22time_grain_sqla%22%3Anull%2C%22since%22%3A%227+days+ago%22%2C%22until%22%3A%22now%22%2C%22entity%22%3A%22country%22%2C%22country_fieldtype%22%3A%22name%22%2C%22metric%22%3A%22count%22%2C%22show_bubbles%22%3Atrue%2C%22secondary_metric%22%3A%22count%22%2C%22max_bubble_size%22%3A%2225%22%2C%22where%22%3A%22%22%2C%22having%22%3A%22%22%2C%22filters%22%3A%5B%5D%7D&standalone=true&height=400"
        >
        </iframe>""")

---
# Рекомендательные системы
---

В этой лабораторной работе будет рассмотрена задача предсказания оценки, которую пользователь поставит фильму. Особенность этой задачи в том, что объекты выборки описываются категориальными признаками, принимающими большое число значений (например: идентификатор пользователя, идентификатор фильма, тэги, киноперсоны).

Будем работать с датасетом MovieLens + IMDb/Rotten Tomatoes. Набор содержит данные о предпочтениях пользователей сервиса рекомендации кинофильмов MovieLens. Пользовательские оценки для фильмов принимают ~~целые~~ значения в интервале от 1 до 5, они записаны в файле user_ratedmovies.dat (а так же в user_ratedmovies-timestamps.dat, где для каждой оценки записана дата и время в формате timestamp), остальные файлы содержат дополнительную информацию о фильмах, которую можно использовать как признаки. Заметьте: кроме оценок (и тегов), про пользователя ничего не известно.

На основании этих данных необходимо построить модель, предсказывающую оценку пользователя фильму, который он еще не смотрел.

# Dataset:

MovieLens + IMDb/Rotten Tomatoes

---

### Description

---

This dataset is an extension of MovieLens10M dataset, published by GroupLeans 
research group.

http://www.grouplens.org 

It links the movies of MovieLens dataset with their corresponding web pages at 
Internet Movie Database (IMDb) and Rotten Tomatoes movie review systems.

http://www.imdb.com 

http://www.rottentomatoes.com 

From the original dataset, only those users with both rating and tagging information 
have been mantained.

---

### Data statistics

---
<table>
<tr style="border-bottom: 2pt solid black;"><th>count</th><th>name</th></tr>
<tr><td>2113</td><td>users</td> </tr>
<tr style="border-bottom: 2pt solid black;"><td>10197</td><td>movies</td></tr>

<tr><td>20</td><td>movie genres</td></tr>
<tr><td>20809</td><td>movie genre assignments</td> </tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 2.040 genres per movie</td></tr>

<tr><td>4060</td><td>directors</td></tr>
<tr><td>95321</td><td>actors</td></tr>
<tr><td></td><td>avg. 22.778 actors per movie</td></tr>
<tr style="border-bottom: 2pt solid black;"><td>72</td><td>countries</td></tr>

<tr><td>10197</td><td>country assignments</td></tr>
<tr><td></td><td>avg. 1.000 countries per movie</td></tr>
<tr><td>47899</td><td>location assignments</td></tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 5.350 locations per movie</td></tr>

<tr><td>13222</td><td>tags</td></tr>
<tr><td>47957</td><td>tag assignments (tas), i.e. tuples [user, tag, movie]</td></tr>
<tr><td></td><td>avg. 22.696 tas per user</td></tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 8.117 tas per movie</td></tr>

<tr><td>855598</td><td>ratings</td></tr>
<tr><td></td><td>avg. 404.921 ratings per user</td></tr>
<tr style="border-bottom: 2pt solid black;"><td></td><td>avg. 84.637 ratings per movie</td></tr>

</table>

---
# Оценивание качества рекомендаций:
---

Выберем некоторого пользователя $u$ и обозначим известные для него рейтинги за $R^u$. В качестве тестовых рейтингов этого пользователя $R^u_{test}$ рассмотрим три рейтинга, поставленные последними по времени. Остальные известные рейтинги будут составлять обучающую выборку $R^u_{train}$. Тогда все известные рейтинги можно представить как $R^u = R^u_{train} \cup R^u_{test}$. Отсутствующие оценки обозначим за $R^u_{unknown}$. Объединив эти наборы для всех пользователей, получим наборы $R_{train}$, $R_{test}$ и $R_{unknown}$.

Для измерения качества рекомендаций в этой лабораторной работе будем использовать две метрики RMSE и MAP, описанные ниже.

---
### RMSE
---
Метрика RMSE вычисляется следующим образом:

\begin{array}
-RMSE & =  &\sqrt{\frac{1}{|R_{test}|}\sum_{(u, i) \in R_{test}} (r_{ui} - \widehat{r}_{ui})^2}
\end{array}

где $r_{ui}$ - наблюдаемая (правильная) оценка, а  $\widehat{r}_{ui}$ - оценка, предсказанная моделью.

Метрика RMSE предназначена для оценки точности предсказания, ее удобно оптимизировать напрямую. Однако, нужно учесть, что RMSE не лучший кандидат для оценки качества рекомендаций:

* Во-первых, RMSE оценивает точность предсказания рейтингов, что в задачах рекомендаций, как правило, менее важно, нежели верное ранжирование объектов (безотносительно абсолютных значений рейтингов). На практике часто бывает, что алгоритмы с худшим RMSE в продакшене работают лучше, нежели алгоритмы с меньшим значением RMSE. Например, при использовании моделей рекомендаций на основе матричных разложений, заполнение отсутствующих оценок нулями даёт худшее значение по RMSE, но при этом часто сами рекомендации становятся лучше.
* Во-вторых, RMSE одинаково штрафует точность предсказания оценок фильмам с большим значением предпочтения (которые попадут в блок рекомендаций) и фильмам с малым значением предпочтения (длинный хвост из нерелевантных фильмов).

---
### MAP
---

Для оценки качества рекомендаций также можно использовать метрику качества ранжирования. Для этого для каждого пользователя $u$ предскажем оценку для всех фильмов из $R^u_{test}$ и $R^u_{unknown}$ и отсортируем эти фильмы по убыванию предсказанного рейтинга. Ожидается, что хороший алгоритм должен выдать релевантные фильмы вверху списка. Обозначим позиции объектов в этом списке за $k^u_{i}$.

Назовем релевантными те фильмы, которые входят в $R^u_{test}$ и имеют оценку $\ge 3$. Обозначим их за $Rel^u$. Тогда можно считать следующую метрику качества рекомендаций для одного пользователя:

\begin{array}
-AP^u = \frac{1}{|Rel^u|} \sum\limits_{(u, i) \in Rel^u} \frac{1}{k_i^u}.
\end{array}

Усреднив значение этой метрики по всем пользователями, мы получим окончательное значение метрики MAP. Пользователей без релевантных фильмов в тестовой выборке можно не учитывать.

---
### Другие способы оцениваний рекомендаций
---

На практике, как правило, качество рекомендательных систем оценивается в онлайне с помощью A/B-тестирования.

In [2]:
def rmse_metric(data):
    return np.sqrt(np.mean([(row['rating'] - row['prediction']) ** 2 
                            for _, row in data.iterrows()]))

def map_metric(data):
    def ap_metric(user_id, movie_ratings):
        movie_indexs = list(movie_ratings.sort_values(by='prediction', ascending=False).index)
        
        value, count_rel = 0., 0
        for movie_id, row in movie_ratings.iterrows():
            if row['rating'] >= 3:
                k = movie_indexs.index(movie_id) + 1
                value += 1. / k
                count_rel += 1
        return value / count_rel if count_rel > 0 else 0.
    
    value, count_rel = 0., 0
    for user_id, movie_ratings in data.reset_index().groupby('userID'):
        t = ap_metric(user_id, movie_ratings.set_index('movieID'))
        
        if t > 0.:
            value += t
            count_rel += 1
    
    return value / count_rel if count_rel > 0 else 0

# Коллаборативная фильтрация

Загрузите данные и создайте матрицу пользователи-фильмы, где в каждой ячейке стоит рейтинг, если он известен, или ноль, если неизвестен.

In [3]:
data = pd.read_table('data/user_ratedmovies-timestamps.dat', encoding='cp1251', index_col=['userID', 'movieID'])
data.head(5)

rating      timestamp
userID movieID                       
75     3           1.0  1162160236000
       32          4.5  1162160624000
       110         4.0  1162161008000
       160         2.0  1162160212000
       163         4.0  1162160970000

In [4]:
in_train = np.array([True] * data.shape[0])

prev_user_id = None
for i, (index, row) in enumerate(data.iterrows()):
    user_id, movie_id = index
    if prev_user_id != user_id:
        prev_user_id = user_id
        border_timestamp = data.ix[user_id].sort_values('timestamp').iloc[-3]['timestamp']
    
    timestamp = row['timestamp']
    if timestamp >= border_timestamp:
        in_train[i] = False

train = data[in_train]
test = data[np.logical_not(in_train)]

R = pd.pivot_table(data.reset_index(), 
                   columns='movieID', 
                   index='userID', 
                   values='rating', 
                   aggfunc=np.mean, 
                   fill_value=0.)

unknown = R.stack().drop(data.index)
unknown = unknown.to_frame(name='rating')

In [6]:
if 1 == 0:
    train.to_csv('data/solution/solution.train.csv')
    test.to_csv('data/solution/solution.test.csv')
    unknown.to_csv('data/solution/solution.unknown.csv')

In [7]:
if 1 == 0:
    train = pd.read_csv('data/solution/solution.train.csv', index_col=['userID', 'movieID'])
    test = pd.read_csv('data/solution/solution.test.csv', index_col=['userID', 'movieID'])
    unknown = pd.read_csv('data/solution/solution.unknown.csv', header=None)
    unknown.columns = ['userID', 'movieID', 'rating']
    unknown = unknown.set_index(['userID', 'movieID'])

## 1. Most popular method

__Задание:__ Постройте рекоммендации на основе __most popular__ метода, при котором пользователям рекомендуются объекты в порядке убываниях их популярности (например, среднего рейтинга). Оцените качество рекомендаций с использованием метрики MAP.

In [12]:
movie_ratings = train.reset_index().groupby('movieID').agg({'rating': np.mean})
movie_ratings.columns = ['prediction']
prediction = pd.concat([test[['rating']], unknown]).join(movie_ratings).fillna(0.)
prediction.head()

rating  prediction
userID movieID                    
75     2571        4.5    4.174834
       5952        3.5    4.032301
       7153        3.5    4.095090
78     8400        4.5    3.656250
       44694       2.0    3.859447

In [13]:
print('MAP: %f' % map_metric(prediction))
print('RMSE: %f' % rmse_metric(prediction))

MAP: 0.001123
RMSE: 3.262988


## 2. Item-based method

__Задание:__ Теперь рассмотрим __memory-based__ методы рекоммендаций. Подход, лежащий в их основе, использует данные о рейтингах для вычисления сходства между пользователями (__user-based__) или объектами (__item-based__), на основе этих данных делаются предсказания рейтингов и, в дальнейшем, строятся рекоммендации. Эти методы просты в реализации и эффективны на ранних стадиях разработки рекомендательных систем.

Постройте рекоммендации на основе item-based подхода, реализованном в библиотеке Graphlab Create. Оцените качество рекомендаций, в зависимости от выбранной функции похожести __jaccard/cosine/pearson__ по каждой из описанных выше метрик (RMSE, MAP).

### Jacard

In [5]:
model = graphlab.recommender.item_similarity_recommender.create(graphlab.SFrame(data=train.reset_index()),
                                                                user_id='userID',
                                                                item_id='movieID',
                                                                target='rating',
                                                                similarity_type='jaccard',
                                                                verbose=False)

prediction = pd.concat([test, unknown])
prediction['prediction'] = model.predict(graphlab.SFrame(data=prediction.reset_index()))

prediction.head()

This non-commercial license of GraphLab Create for academic use is assigned to fpm.yunusov@bsu.by and will expire on May 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1494574077.log


Recsys training: model = item_similarity

rating     timestamp  prediction
userID movieID                                  
75     2571        4.5  1.162161e+12    0.196205
       5952        3.5  1.162161e+12    0.163237
       7153        3.5  1.162161e+12    0.159916
78     8400        4.5  1.177224e+12    0.000350
       44694       2.0  1.179550e+12    0.000732

In [6]:
print('MAP: %f' % map_metric(prediction))
print('RMSE: %f' % rmse_metric(prediction))

MAP: 0.019299
RMSE: 0.066082


### Cosine

In [7]:
model = graphlab.recommender.item_similarity_recommender.create(graphlab.SFrame(data=train[['rating']].reset_index()),
                                                                user_id='userID',
                                                                item_id='movieID',
                                                                target='rating',
                                                                similarity_type='cosine',
                                                                verbose=False)

prediction = pd.concat([test, unknown])
prediction['prediction'] = model.predict(graphlab.SFrame(data=prediction.reset_index()))

prediction.head()

Recsys training: model = item_similarity

rating     timestamp  prediction
userID movieID                                  
75     2571        4.5  1.162161e+12    1.120120
       5952        3.5  1.162161e+12    1.030550
       7153        3.5  1.162161e+12    0.934922
78     8400        4.5  1.177224e+12    0.002233
       44694       2.0  1.179550e+12    0.005015

In [8]:
print('MAP: %f' % map_metric(prediction))
print('RMSE: %f' % rmse_metric(prediction))

MAP: 0.022464
RMSE: 0.083065


### Pearson

In [9]:
model = graphlab.recommender.item_similarity_recommender.create(graphlab.SFrame(data=train[['rating']].reset_index()),
                                                                user_id='userID',
                                                                item_id='movieID',
                                                                target='rating',
                                                                similarity_type='pearson',
                                                                verbose=False)

prediction = pd.concat([test, unknown])
prediction['prediction'] = model.predict(graphlab.SFrame(data=prediction.reset_index()))

prediction.head()

Recsys training: model = item_similarity

rating     timestamp  prediction
userID movieID                                  
75     2571        4.5  1.162161e+12    4.179417
       5952        3.5  1.162161e+12    4.024876
       7153        3.5  1.162161e+12    4.088363
78     8400        4.5  1.177224e+12    3.658329
       44694       2.0  1.179550e+12    3.859334

In [10]:
print('MAP: %f' % map_metric(prediction))
print('RMSE: %f' % rmse_metric(prediction))

MAP: 0.001123
RMSE: 3.263030


## Разреженный SVD

__Задание:__ Разложите матрицу рейтингов с помощью разреженного SVD и, восстановив ее, получите предсказания рейтингов для всех пар пользователь-объект (этот метод, при котором неизвестные рейтинги заполняются нулями, а затем восстанавливаются с помощью SVD, называется PureSVD).

In [15]:
def run_svd(n_components):
    global R, train, test, unknown
    model = TruncatedSVD(n_components=n_components)

    model.fit(R)
    new_R = pd.DataFrame(data=np.dot(model.transform(R), model.components_),
                         index=R.index,
                         columns=R.columns)
    new_R = pd.concat([test, unknown]).join(new_R.stack().to_frame('prediction').drop(train.index))
    map_m = map_metric(new_R)
    rmse_m = rmse_metric(new_R)
#     print('I: %i\tMAP: %f\tRMSE: %f' % (i, map_m, rmse_m))
    return (n_components, map_m, rmse_m)

In [22]:
run_svd(20)

I: 15	MAP: 0.038278	RMSE: 0.283128


(20, 0.03827811065660048, 0.28312843019200201)

__Задание:__ Проведите эксперименты из предыдущего задания для алгоритма рекомендаций RankingFactorizationRecommender из GraphLab Create, сравните его с результатами полученными ранее.

In [19]:
model = graphlab.recommender.ranking_factorization_recommender.create(graphlab.SFrame(data=train[['rating']].reset_index()),
                                                                      user_id='userID',
                                                                      item_id='movieID',
                                                                      target='rating',
                                                                      num_factors=20,
                                                                      verbose=False)

prediction = pd.concat([test, unknown])
prediction['prediction'] = model.predict(graphlab.SFrame(data=prediction.reset_index()))

prediction.head()

Recsys training: model = ranking_factorization_recommender

rating     timestamp  prediction
userID movieID                                  
75     2571        4.5  1.162161e+12    4.472067
       5952        3.5  1.162161e+12    4.217894
       7153        3.5  1.162161e+12    4.251242
78     8400        4.5  1.177224e+12    3.256014
       44694       2.0  1.179550e+12    3.456278

In [20]:
print('MAP: %f' % map_metric(prediction))
print('RMSE: %f' % rmse_metric(prediction))

MAP: 0.026134
RMSE: 1.675873


__Задание:__ Для алгоритма из предыдущего пункта рассмотрите вместо квадратичных потерь логистические, т.е. решите задачу бинарной классификации (отделение хороших фильмов от плохих). Для этого вам понадобится использовать параметр binary_targets. Дал ли этот подход прирост в качестве?

In [21]:
model = graphlab.recommender.ranking_factorization_recommender.create(graphlab.SFrame(data=train[['rating']].reset_index()),
                                                                      user_id='userID',
                                                                      item_id='movieID',
                                                                      target='rating',
                                                                      num_factors=20,
                                                                      binary_target=True,
                                                                      verbose=False)

prediction = pd.concat([test, unknown])
prediction['prediction'] = model.predict(graphlab.SFrame(data=prediction.reset_index()))

prediction.head()

Recsys training: model = ranking_factorization_recommender

[ERROR] graphlab.toolkits._main: Toolkit error: Training with binary_target=True requires targets to be 0 or 1; (4.500000 invalid).


ToolkitError: Training with binary_target=True requires targets to be 0 or 1; (4.500000 invalid).

In [ ]:
print('MAP: %f' % map_metric(prediction))
print('RMSE: %f' % rmse_metric(prediction))

__Задание:__ Поэксперементируйте с следующими четырьмя параметрами алгоритма RankingFactorizationRecommender:

* num_factors
* ranking_regularization
* unobserved_rating_value
* num_sampled_negative_examples